# Census and Zillow Data Cleaning
After talking with Tom and John from MAHA and GBIO, one ask that came to us was to see if there were any ways we could use data to get insight into the "NIMBY vs. YIMBY" debate currently happening in housing justice communities across the U.S.

When we started talking about what data could provide value into this conversation, we thought about rental data, and Zillow came up as a possible source of data. We also thought the census would have useful data to do this research too. Both of these sources of data turned out to be useful as a starting point for the conversation.

From the census, we were able to find data that contained information on the number of authorized construction permits for private residential units, per year, per metro area (as defined by the census). This was retrievable via the [FactFinder](https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml) area of their website. There was data going as far back as the early 2000s, but all of the data collected after 2014 was collected differently – so I started by just looking at data for 2014-2016.

Zillow also has a set of [research data sets](https://www.zillow.com/research/data/). From this, I was able to find median rent prices, per year, per metro area (as defined by Zillow). The Zillow data had median rent prices broken down as far as neighborhoods within metro areas (i.e. Queens within New York), so I had to consolidate those neighborhoods first, and grab the average median rent per metro area.

You can see both of these data sets in our google drive.

With both of these data sets, it was a matter of cleaning and combining the data, trying as best as possible to match the data by similarly names metro regions. The formats for each metro region were different, but for the most part, Zillow would use the central city of a metro area to refer to the whole region, while the census would use a more detailed name (i.e. "Lost Angeles" in Zillow, and "Los Angeles-Aneheim" in the census). So, in order to match them, I basically searched each census metro area's name to see if it contained the name of the Zillow metro area or not – this means asking through code to see if the words "Los Angeles" appear anywhere in the phrases "New York-Newark", or in "Los Angeles-Aneheim", and putting the data from one data set alongside the other if they do.

Here's some of the code I used to get the data from a `.csv` format, and combine it into one data set, and export it as its own `.csv` file.

```python
# first, consolidate the Zillow median rent data on 2BR houses into one file of median rent per metro area
try:
    with open('2BR-median-rent-data.csv', 'rU') as readfile: # Zillow median rent data 2BR, by neighborhood
        reader = csv.DictReader(readfile, dialect=csv.excel) # Library for parsing csv files into python
        for row in reader:
            if row['Metro Area 3BR'] != '': # I mislabeled the data when I pulled it down by accident
                key = '%s-%s' % (row['Metro Area 3BR'].lower(), row['State 3BR'].lower())
                if two_br_data.get(key) is None:
                    two_br_data[key] = {'state': row['State 3BR'].lower(),'2014': [],'2015': [],'2016': []}
                if row['3BR 2014 AVG Median Rent'] != '':
                    two_br_data[key]['2014'].append(row['3BR 2014 AVG Median Rent'])
                if row['3BR 2015 AVG Median Rent'] != '':
                    two_br_data[key]['2015'].append(row['3BR 2015 AVG Median Rent'])
                if row['3BR 2016 AVG Median Rent'] != '':
                    two_br_data[key]['2016'].append(row['3BR 2016 AVG Median Rent'])
except OSError:
    print ("error getting input file")
finally:
    readfile.close()

with open('2BR-median-rent-data-consolidated.csv', 'w') as writefile: # The file to create, with median rent prices per metro area
    writer = csv.DictWriter(writefile, fieldnames=['State', 'Metro', '2014 AVG Median Rent', '2015 AVG Median Rent', '2016 AVG Median Rent'])
    writer.writeheader()
    for key in two_br_data.keys():
        median_2014 = ''
        median_2015 = ''
        median_2016 = ''
        if len(two_br_data[key]['2014']) > 0: # create average median rent prices/year for each metro area
            median_2014 = functools.reduce(lambda x, y: x + float(y), two_br_data[key]['2014'], 0) / len(two_br_data[key]['2014'])
        if len(two_br_data[key]['2015']) > 0:
            median_2015 = functools.reduce(lambda x, y: x + float(y), two_br_data[key]['2015'], 0) / len(two_br_data[key]['2015'])
        if len(two_br_data[key]['2016']) > 0:
            median_2016 = functools.reduce(lambda x, y: x + float(y), two_br_data[key]['2016'], 0) / len(two_br_data[key]['2016'])
        new_row = {'State': two_br_data[key]['state'], 'Metro': key.split('-')[0], '2014 AVG Median Rent': median_2014, '2015 AVG Median Rent': median_2015, '2016 AVG Median Rent': median_2016}
        writer.writerow(new_row)

# consolidate data in the same way, but for 3BR houses on Zillow
try:
    with open('3BR-median-rent-data.csv', 'rU') as readfile:
        reader = csv.DictReader(readfile, dialect=csv.excel)
        for row in reader:
            if row['Metro'] != '':
                key = '%s-%s' % (row['Metro'].lower(), row['State'].lower())
                if three_br_data.get(key) is None:
                    three_br_data[key] = {'state': row['State'].lower(),'2014': [],'2015': [],'2016': []}
                if row['2014-average'] != '':
                    three_br_data[key]['2014'].append(row['2014-average'])
                if row['2015-average'] != '':
                    three_br_data[key]['2015'].append(row['2015-average'])
                if row['2016-average'] != '':
                    three_br_data[key]['2016'].append(row['2016-average'])
except OSError:
    print ("error getting input file")
finally:
    readfile.close()

with open('3BR-median-rent-data-consolidated.csv', 'w') as writefile:
    writer = csv.DictWriter(writefile, fieldnames=['State', 'Metro', '2014 AVG Median Rent', '2015 AVG Median Rent', '2016 AVG Median Rent'])
    writer.writeheader()
    for key in three_br_data.keys():
        median_2014 = ''
        median_2015 = ''
        median_2016 = ''
        if len(three_br_data[key]['2014']) > 0:
            median_2014 = functools.reduce(lambda x, y: x + float(y), three_br_data[key]['2014'], 0) / len(three_br_data[key]['2014'])
        if len(three_br_data[key]['2015']) > 0:
            median_2015 = functools.reduce(lambda x, y: x + float(y), three_br_data[key]['2015'], 0) / len(three_br_data[key]['2015'])
        if len(three_br_data[key]['2016']) > 0:
            median_2016 = functools.reduce(lambda x, y: x + float(y), three_br_data[key]['2016'], 0) / len(three_br_data[key]['2016'])
        new_row = {'State': three_br_data[key]['state'], 'Metro': key.split('-')[0], '2014 AVG Median Rent': median_2014, '2015 AVG Median Rent': median_2015, '2016 AVG Median Rent': median_2016}
        writer.writerow(new_row)


# Create a dictionary in python into which to put the consolidated data
data_together = {}

# First, get the data for the 3BR houses from the file created to hold average median rent prices by area
try:
    with open('3BR-median-rent-data-consolidated.csv', 'rU') as readfile:
        reader = csv.DictReader(readfile, dialect=csv.excel)
        for row in reader:
            key = '%s-%s' % (row['Metro'], row['State'])
            if data_together.get(key) is None:
                data_together[key] = {'state': row['State'],'3BR2014': row['2014 AVG Median Rent'],'3BR2015': row['2015 AVG Median Rent'],'3BR2016': row['2016 AVG Median Rent']}
except OSError:
    print ("error getting input file")
finally:
    readfile.close()

# Next add in the data from 2BR houses – metro area names are formatted the same way for both of these data sets,
# because they're both from Zillow
try:
    with open('2BR-median-rent-data-consolidated.csv', 'rU') as readfile:
        reader = csv.DictReader(readfile, dialect=csv.excel)
        for row in reader:
            # Check if the given metro area for the given state is in `data_together` already. If not, add it.
            key = '%s-%s' % (row['Metro'], row['State'])
            if data_together.get(key) is None:
                data_together[key] = {'state': row['State'],'2BR2014': row['2014 AVG Median Rent'],'2BR2015': row['2015 AVG Median Rent'],'2BR2016': row['2016 AVG Median Rent']}
            else:
                data_together[key].update({'2BR2014': row['2014 AVG Median Rent'],'2BR2015': row['2015 AVG Median Rent'],'2BR2016': row['2016 AVG Median Rent']})
except OSError:
    print ("error getting input file")
finally:
    readfile.close()

# Now, open the data from the census on authorized construction permits for new housing to add it in
try:
    with open('private-units-authorized-consolidated.csv', 'rU') as readfile:
        reader = csv.DictReader(readfile, dialect=csv.excel)
        for row in reader:
            # look at every metro area we already have stored from the Zillow data. If the metro area name we have from
            # the Zillow data is also within the metro area name from the census data, add the census data to it.
            for key in data_together.keys():
                metro = key.split('-')[0]
                state = key.split('-')[1]
                if row['Metro'].lower().find(metro) != -1 and row['Metro'].split(',')[1].lower().find(state) != -1:
                    data_together[key].update({'Census_Metro': row['Metro']})
                    for col in row.keys():
                        if col != 'Metro':
                            data_together[key][col] = row[col]
                    break
except OSError:
    print ("error getting input file")
finally:
    readfile.close()

# Write the combined Zillow/Census data to a new file that we can now look at in Google Sheets or Excel
with open('authorized-rent-consolidated.csv', 'w') as writefile:
    fieldnames = ['2014 3 and 4 Units', '2016 3 and 4 Units', '3BR2014', '2BR2015', '2016 Units Authorized', '2014 2 Units', '2016 2 Units', '2015 2 Units', '2015 Units Authorized', '2015 3 and 4 Units', '2014 Units Authorized', '3BR2016', 'Census_Metro', '3BR2015', 'state', '2BR2014', '2BR2016']
    writer = csv.DictWriter(writefile, fieldnames=['2014 3 and 4 Units', '2016 3 and 4 Units', '3BR2014', '2BR2015', '2016 Units Authorized', '2014 2 Units', '2016 2 Units', '2015 2 Units', '2015 Units Authorized', '2015 3 and 4 Units', '2014 Units Authorized', '3BR2016', 'Census_Metro', '3BR2015', 'state', 'metro', '2BR2014', '2BR2016'])
    writer.writeheader()
    for key in data_together.keys():
        row = {}
        row['metro'] = key.split('-')[0]
        for field in fieldnames:
            row[field] = data_together[key].get(field, '')
        writer.writerow(row)
```

After adding in the dataset on authorized construction, I found another data set from the ACS that has yearly housing statistics for metro areas across the U.S. This contains information on vacancy rates within a metro area, what kind of housing units are in a metro area (i.e. if there are apartment buildings or standalone houses, etc.), what the estimated value of housing is, and a lot more. Combined this using the folloiwng code:

```python
try:
    with open('/Users/Matt/Documents/sunshine-b/ACS_16_1YR_CP04_with_ann.csv', 'rU') as readfile:
        reader = csv.DictReader(readfile, dialect=csv.excel)
        for row in reader:
            for key in all_data.keys():
                metro = key.split('-')[0]
                state = key.split('-')[1]
                if row['Geography'].lower().find(metro) != -1 and row['Geography'].split(',')[1].lower().find(state) != -1:
                    all_data[key].update(row)
                    break
except OSError:
    print ("error getting input file")
finally:
    readfile.close()
```

Adding in all of the data from the ACS survey made the number of data points per metro area untenable. So, I decided to build two different data sets from all of this. 

### Data Set 1: Percentages Only
The first one just kept all data on metro areas that was percentage-based – this means rather than having the "number of all newly authorized housing units", converting that into "the number of newly authorized units as a percentage of the total existing housing units". It also means keeping the data point on "vacancy rate" for a metro area, but not the total number of vacant units. While total values would be helpful data points for analyzing different trends in big vs. small metro areas, this seemed like a useful starting point for looking at the huge number of data points.

### Data Set 2: Statistically Significant Changes
One cool thing about the ACS data is that it had data points which marked whether the change in data points from year to year were statistically significant or not. What this means: if, say, the percentage of rentors paying more than 35% of their income to rent grew by 1%, a data point would be marked saying that there was a significant change from 2015 to 2016 in that data point. If it changed by, say, 0.01%, the data point would say that there was no statistically significant change. This kind of True/False data point would be useful to analyze when looking at trends in cities from 2014 to 2016.

These data sets were edited/aggregated in Excel rather than by code.